<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week7.do.txt --no_mako -->
<!-- dom:TITLE: March 6-10, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# March 6-10, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University

Date: **March 6-10**

## Algorithms for solving quantum mechanical problems, quantum gates and circuits

1. Introduction to [Qiskit](https://qiskit.org/)

2. Defining one-, two- and three-qubit gates

3. Setting up quantum circuits and simple algorithms

4. VQE, Variational Quantum Eigensolver

5. Selected Hamiltonians

## Quantum gates, reminder from last week

## Introduction to [Qiskit](https://qiskit.org/)

        #!/usr/bin/env python
        # coding: utf-8
        import numpy as np
        import qiskit as qk
        from scipy.optimize import minimize
        
        
        # # Initialize registers and circuit
        
        n_qubits = 1 #Number of qubits
        n_cbits = 1 #Number of classical bits (the number of qubits you want to measure at the end of the circuit)
        qreg = qk.QuantumRegister(n_qubits) #Create a quantum register
        creg = qk.ClassicalRegister(n_cbits) #Create a classical register
        circuit = qk.QuantumCircuit(qreg,creg) #Create your quantum circuit
        
        
        circuit.draw() #Draw circuit. It is empty
        
        
        # # Perform operations on qubit
        
        
        circuit.x(qreg[0]) #Applies a Pauli X gate to the first qubit in the quantum register
        circuit.draw()
        
        
        # # Chose a qubit to measure and encode the results to a classical bit
        
        
        
        #Measure the first qubit in the quantum register
        #and encode the results to the first qubit in the classical register
        circuit.measure(qreg[0],creg[0])
        circuit.draw()
        
        
        # # Execute circuit
        
        
        
        backend = qk.Aer.get_backend('qasm_simulator') 
        #This is the device you want to use. It is an ideal simulation of a quantum device
        
        
        job = backend.run(circuit,shots=1000) #Run the circuit 1000 times
        result = job.result()
        counts = result.get_counts()
        print(counts)
        circuit.clear()
        circuit.draw()
        
        
        circuit.h(qreg[0]) #Apply a Hadamard gate to the first qubit of the quantum register
        circuit.measure(qreg,creg)
        print(circuit.draw())
        
        
        job = backend.run(circuit,shots=1000)
        result = job.result()
        counts = result.get_counts()
        print(counts)
        circuit.clear()
        
        
        # # Create a two-qubit circuit and set up a Bell state
        
        
        
        n_qubits = 2
        n_cbits = 2
        qreg = qk.QuantumRegister(n_qubits)
        creg = qk.ClassicalRegister(n_cbits)
        circuit = qk.QuantumCircuit(qreg,creg)
        circuit.draw()
        
        
        
        circuit.h(qreg[0])
        circuit.cx(qreg[0],qreg[1]) 
        #This is a controlled operation. Apply a Pauli X gate to the second qubit (qreg[1]) if the first qubit (qreg[0])
        #is in the |1> state. Else do nothing
        
        circuit.draw()
        
        
        
        
        circuit.measure(qreg,creg)
        circuit.draw()
        
        
        # In[12]:
        
        
        job = backend.run(circuit,shots=1000)
        result = job.result()
        counts = result.get_counts()
        print(counts)
        circuit.clear()
        
        
        # # Apply rotation to qubit
        
        
        theta = np.pi/3
        circuit.rx(theta, qreg[0]) #R_x(theta) rotation on the first qubit (qreg[0])
        circuit.measure(qreg,creg)
        print(circuit.draw())
        job = backend.run(circuit,shots=1000)
        result = job.result()
        counts = result.get_counts()
        circuit.clear()
        print(counts)
        
        
        # # Find the lowest eigenvalue of $$ H = c_1 Z_0 + c_2 Z_1 + c_3 X_0 Y_1 $$ 
        # # We will use $$<\psi|H|\psi> = c_1<\psi|Z_0|\psi> + c_2<\psi|Z_1|\psi> + c_3<\psi|X_0Y_1|\psi> $$
        
        
        
        I = np.eye(2)
        X = np.array([[0,1],[1,0]])
        Y = np.array([[0,-1j],[1j,0]])
        Z = np.array([[1,0],[0,-1]])
        H = np.kron(Z,I) + np.kron(I,Z) + np.kron(X,Y)
        eigvals,eigvecs = np.linalg.eigh(H)
        print(eigvals[0])
        
        
        
        c_1 = 1
        c_2 = 1
        c_3 = 1
        
        h_1 = [c_1,[0],['z']]
        h_2 = [c_2,[1],['z']]
        h_3 = [c_3,[0,1],['x','y']]
        H = [h_1,h_2,h_3]
        H
        
        
        
        H[0]
        
        
        # # Create ansatz
        
        
        
        def ansatz(theta,n_qubits):
            qreg = qk.QuantumRegister(n_qubits)
            circuit = qk.QuantumCircuit(qreg)
            for i in range(n_qubits):
                circuit.ry(theta[i],qreg[i])
            for i in range(n_qubits-1):
                circuit.cx(qreg[i],qreg[i+1])
            return(circuit)
        qreg = qk.QuantumRegister(n_qubits)
        circuit = qk.QuantumCircuit(qreg)
        circuit.h(qreg[:2])
        print('Before ansatz')
        print(circuit.draw())
        theta = np.random.randn(2)
        n_qubits = 2
        circuit = circuit.compose(ansatz(theta,n_qubits))
        print('After ansatz')
        circuit.draw()
        
        
        # # Change measurement basis
        
        
        
        def basis_change(h_i,n_qubits):
            qreg = qk.QuantumRegister(n_qubits)
            circuit = qk.QuantumCircuit(qreg)
            
            for qubit,operator in zip(h_i[1],h_i[2]):
                if operator == 'x':
                    circuit.h(qreg[qubit])
                if operator == 'y':
                    circuit.sdg(qreg[qubit])
                    circuit.h(qreg[qubit])
            return(circuit)
        n_qubits = 2
        qreg = qk.QuantumRegister(n_qubits)
        circuit = qk.QuantumCircuit(qreg)
        theta = np.random.randn(n_qubits)
        circuit = circuit.compose(ansatz(theta,n_qubits))
        print('Ansatz circuit')
        circuit.draw()
        circuit = circuit.compose(basis_change(H[2],n_qubits))
        print('After basis transformation:')
        print(circuit.draw())
        
                
        
        
        # # Get energy for given rotational parameters, theta
        
        
        
        def get_energy(theta):
            n_qubits = 2
            qreg = qk.QuantumRegister(n_qubits)
            circuit = qk.QuantumCircuit(qreg)
            circuit = circuit.compose(ansatz(theta,n_qubits))
            circuit_list = []
            for idx,h_i in enumerate(H):
                basis_change_circuit = basis_change(h_i,n_qubits)
                new_circuit = circuit.compose(basis_change_circuit)
                creg = qk.ClassicalRegister(len(h_i[1]))
                new_circuit.add_register(creg)
                new_circuit.measure(qreg[h_i[1]],creg)
                circuit_list.append(new_circuit)
            shots = 10000
            job = backend.run(circuit_list,shots=shots)
            E = np.zeros(len(circuit_list))
            for i in range(len(circuit_list)):
                result = job.result()
                counts = result.get_counts(i)
                for key,value in counts.items():
                    e = 1
                    for bit in key:
                        if bit == '0':
                            e *= 1
                        if bit == '1':
                            e *= -1
                    E[i] += e*value
                E[i] *= H[i][0]
            E /= shots
            return(np.sum(E))
        
        theta = np.random.randn(2)
        get_energy(theta)
            
        
        
        # # Minimize energy with Scipy
        
        
        
        
        theta = np.random.randn(2)
        res = minimize(get_energy, theta, method='Powell',tol=1e-12)
        get_energy(res.x)
        
        
        # ## We might need a more flexible ansatz
        
        
        
        
        def ansatz(theta,n_qubits):
            qreg = qk.QuantumRegister(n_qubits)
            circuit = qk.QuantumCircuit(qreg)
            idx = 0
            for i in range(n_qubits):
                circuit.ry(theta[idx],qreg[i])
                idx += 1
            for i in range(n_qubits-1):
                circuit.cx(qreg[i],qreg[i+1])
            for i in range(n_qubits):
                circuit.rx(theta[idx],qreg[i])
                idx += 1
            for i in range(n_qubits-1):
                circuit.cx(qreg[i],qreg[i+1])
            return(circuit)
        theta = np.random.randn(4)
        res = minimize(get_energy, theta, method='Powell',tol=1e-16)
        get_energy(res.x)
        
        
        # # Minimize energy with gradient descent
        # # $$ \frac{\partial E (\theta_1,\dots,\theta_i,\dots,\theta_p)}{\partial \theta_i} = \frac{E(\theta_1,\dots,\theta_i + \pi/2,\dots, \theta_p) - E(\theta_1,\dots, \theta_i - \pi/2,\dots, \theta_p}{2} $$
        
        
        
        epochs = 200
        theta = np.random.randn(4)
        for epoch in range(epochs):
            print(epoch,get_energy(theta))
            grad = np.zeros_like(theta)
            for idx in range(theta.shape[0]):
                theta_temp = theta.copy()
                theta_temp[idx] += np.pi/2
                E_plus = get_energy(theta_temp)
                theta_temp[idx] -= np.pi
                E_minus = get_energy(theta_temp)
                grad[idx] = (E_plus - E_minus)/2
            theta -= 0.1*grad
        
